In [1]:
import numpy as np
import json
import torch

In [2]:
import matplotlib.pyplot as plt

In [3]:
from survey_ops.coreRL.data_processing import load_raw_data_to_dataframe
from survey_ops.coreRL.offline_dataset import OfflineDECamDataset
from survey_ops.utils import ephemerides

# Load experiment

In [4]:
exp_dir = '../experiment_results/ddqn-azel-best/'
with open(exp_dir + 'config.json', 'r') as f:
    cfg = json.load(f)
with open('../configs/global_config.json', 'r') as f:
    gcfg = json.load(f)

In [5]:
df = load_raw_data_to_dataframe(gcfg['paths']['FITS_DIR'] + gcfg['files']['DECFITS'], gcfg['paths']['FITS_DIR'] + gcfg['files']['DECJSON'])

In [8]:
dset = OfflineDECamDataset(
    df=df,
    cfg=cfg,
    glob_cfg=gcfg
    )

Calculating zenith states: 100%|█████████████████████████████████████████████████████| 21/21 [00:00<00:00, 339.14it/s]
Calculating bin features for all healpix bins and timestamps: 100%|██████████| 1942/1942 [00:00<00:00, 3786768.19it/s]
Normalizing bin features: 0it [00:00, ?it/s]


AttributeError: 'OfflineDECamDataset' object has no attribute 'reward_choice'

In [ ]:
plt.plot(dset.rewards, marker='o')

In [14]:
dset._df.iloc[dset.next_state_idxs-1]

,expnum,ra,dec,exptime,filter,propid,program,object,teff,fwhm,...,ha_cos,ha_sin,sun_ra_cos,sun_ra_sin,sun_az_cos,sun_az_sin,moon_ra_cos,moon_ra_sin,moon_az_cos,moon_az_sin
1921,NaN,0.197366,-0.526560,NaN,NaN,NaN,NaN,zenith,NaN,NaN,...,1.000000,0.000000,-0.372732,-0.927939,-0.597078,-0.802183,-0.992176,-0.124847,-0.988181,0.153291
0,798544.0,1.086483,-0.796372,90.0,Y,2012B-0001,survey,DES survey hex 612-461 tiling 9,0.79,0.89,...,0.631577,-0.775313,-0.372730,-0.927940,-0.597449,-0.801907,-0.992171,-0.124883,-0.987956,0.154733
1,798545.0,0.419586,-0.920902,90.0,r,2012B-0001,survey,DES survey hex 241-532 tiling 10,1.25,0.83,...,0.978845,-0.204603,-0.372688,-0.927957,-0.605078,-0.796166,-0.992080,-0.125611,-0.982908,0.184100
2,798546.0,0.460514,-0.903197,90.0,r,2012B-0001,survey,DES survey hex 265-521 tiling 10,1.06,0.92,...,0.971703,-0.236206,-0.372664,-0.927966,-0.609559,-0.792740,-0.992026,-0.126037,-0.979572,0.201096
3,798547.0,0.429452,-0.956315,90.0,r,2012B-0001,survey,DES survey hex 247-552 tiling 10,0.85,1.05,...,0.980358,-0.197225,-0.372639,-0.927976,-0.614015,-0.789295,-0.991972,-0.126459,-0.975995,0.217795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,807658.0,0.987029,-1.149311,90.0,i,2012B-0001,survey,DES survey hex 577-663 tiling 6,0.72,0.95,...,0.766903,0.641763,0.178109,-0.984011,-0.996859,-0.079192,-0.846694,-0.532080,-0.335547,0.942023
1916,807659.0,1.039591,-1.150495,90.0,i,2012B-0001,survey,DES survey hex 577-663 tiling 9,0.68,0.98,...,0.794203,0.607653,0.178136,-0.984006,-0.997606,-0.069161,-0.846504,-0.532382,-0.330881,0.943672
1917,807660.0,1.047130,-1.150789,90.0,i,2012B-0001,survey,DES survey hex 577-663 tiling 7,0.68,0.97,...,0.793855,0.608108,0.178162,-0.984001,-0.998241,-0.059279,-0.846318,-0.532678,-0.326312,0.945262
1918,807661.0,1.145475,-1.151089,90.0,i,2012B-0001,survey,DES survey hex 657-663 tiling 10,0.72,0.93,...,0.844400,0.535713,0.178190,-0.983996,-0.998809,-0.048800,-0.846121,-0.532990,-0.321494,0.946912


In [9]:
np.where(dset.rewards > .4)[0]

array([   0,  208,  408,  587,  702,  853, 1045, 1148, 1436, 1512, 1593,
       1741])

In [10]:
dset.rewards.shape

torch.Size([1903])

In [64]:
dset._df['teff'].values[~np.isnan(dset._df['teff'].values)]

array([0.79, 1.25, 1.06, ..., 0.72, 0.96, 0.81],
      shape=(1921,), dtype=float32)

In [23]:
dset.hpGrid.lon

array([0.78539816, 2.35619449, 3.92699082, ..., 5.988661  , 6.08683577,
       6.18501054], shape=(1504,))

In [28]:
np.count_nonzero(dset.action_masks[10])

np.int64(1496)

In [38]:
np.where(dset.hpGrid.lat < 0)

(array([], dtype=int64),)

In [34]:
np.where(dset.hpGrid.lon == 0)

(array([ 544,  672,  800,  928, 1056, 1184, 1312, 1440]),)

In [32]:
np.where(dset.hpGrid.lon > 0)

(array([   0,    1,    2, ..., 1501, 1502, 1503], shape=(1496,)),)

In [25]:
dset.action_masks.shape

torch.Size([1903, 1504])